# Data from Basketball Reference

## Load the games datasets

In [54]:
import pandas as pd
import numpy as np

In [100]:
games2013 = pd.read_csv('../data/game_outcomes/nba_2013_games.csv')
games2013.shape

(1230, 11)

In [101]:
games2014 = pd.read_csv('../data/game_outcomes/nba_2014_games.csv')
games2014.shape

(1230, 11)

In [102]:
games2015 = pd.read_csv('../data/game_outcomes/nba_2015_games.csv')
games2015.shape

(1230, 11)

In [103]:
games2016 = pd.read_csv('../data/game_outcomes/nba_2016_games.csv')
games2016.shape

(1230, 11)

In [104]:
games2017 = pd.read_csv('../data/game_outcomes/nba_2017_games.csv')
games2017.shape

(1230, 11)

In [105]:
games2018 = pd.read_csv('../data/game_outcomes/nba_2018_games.csv')
games2018.shape

(1230, 11)

In [106]:
games2019 = pd.read_csv('../data/game_outcomes/nba_2019_games.csv')
games2019.shape

(1059, 11)

In [107]:
games2020 = pd.read_csv('../data/game_outcomes/nba_2020_games.csv')
games2020.shape

(1080, 11)

In [108]:
games2021 = pd.read_csv('../data/game_outcomes/nba_2021_games.csv')
games2021.shape

(1230, 11)

## Exploring each game dataset

In [109]:
# Check the dataset
games2013.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes
0,Tue Oct 29 2013,7:00p,Orlando Magic,87,Indiana Pacers,97,Box Score,NaN,18165,Bankers Life Fieldhouse,NaN
1,Tue Oct 29 2013,8:00p,Chicago Bulls,95,Miami Heat,107,Box Score,NaN,19964,AmericanAirlines Arena,NaN
2,Tue Oct 29 2013,10:30p,Los Angeles Clippers,103,Los Angeles Lakers,116,Box Score,NaN,18997,STAPLES Center,NaN
3,Wed Oct 30 2013,7:00p,Brooklyn Nets,94,Cleveland Cavaliers,98,Box Score,NaN,20562,Quicken Loans Arena,NaN
4,Wed Oct 30 2013,7:00p,Boston Celtics,87,Toronto Raptors,93,Box Score,NaN,20155,Air Canada Centre,NaN


In [110]:
# What are the column types?
games2013.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             1230 non-null   object
 1   Start (ET)       1230 non-null   object
 2   Visitor/Neutral  1230 non-null   object
 3   PTS              1230 non-null   int64 
 4   Home/Neutral     1230 non-null   object
 5   PTS.1            1230 non-null   int64 
 6   Unnamed: 6       1230 non-null   object
 7   Unnamed: 7       79 non-null     object
 8   Attend.          1230 non-null   int64 
 9   Arena            1230 non-null   object
 10  Notes            1 non-null      object
dtypes: int64(3), object(8)
memory usage: 105.8+ KB


In [111]:
# What are the unique values in Unnamed: 7?
games2013['Unnamed: 7'].unique()

array([nan, 'OT', '2OT', '3OT'], dtype=object)

In [112]:
# What are the unique values in Notes?
games2013['Notes'].unique()

array([nan, 'at London England'], dtype=object)

From the analysis above, the columns `Start (ET)`, `Unnamed: 6`, `Unnamed: 7`, `Attend.`, `Arena` and `Notes` can be dropped

## Cleaning game datasets

In [113]:
# We are not really interested on the exact data, just the season
def get_season(df):
    """Function to get the season (year) for a pandas DataFrame with NBA game data.
    
    Example:
        For the 2013-2014, the date column is converted to a DateTime object. The year is extracted, 
        and the first entry is used as the year for that season (in this case, 2013).
        
    Args: 
        pandas DataFrame with date object column
    
    Returns: 
        processed DataFrame with season column containing only a year
    
    """
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.year
    df['Season'] = df['Date'][0]
    df.drop('Date', axis=1, inplace=True)
    
    return df.head()

In [114]:
def get_winner(df):
    """Compare if the home or visitor team won the game by scoring checking who scored more points. 
    Output is 1 for home team win, and 0 for away team win. Raw points columns are dropped as a result.
    
    Args: 
        pandas DataFrame with points scored by home and visitor teams.
        
    Returns:
        New column 'H_win' with value 1 when home team wins, and 0 when away team wins. The raw points
        columns for the home and visitor team are dropped for clarity.
    
    """
    df['H_win'] = np.where(df['PTS'] < df['PTS.1'], 1, 0)
    df.drop(['PTS', 'PTS.1'], axis=1, inplace=True)
    
    return df

In [115]:
# Remove all columns that do not include important data
def clean_game_stats(df):
    """Remove unwanted columns from pandas DataFrame with NBA game data.
    
    Args:
        pandas Dataframe.
        
    Returns:
        Cleaned DataFrame without unwanted columns.
    
    """
    df.drop(['Start (ET)', 'Unnamed: 6', 'Unnamed: 7', 'Attend.', 'Arena', 'Notes'], 
            axis=1, inplace=True)
    
    return df

In [116]:
def combine_seasons(df_list):
    """Function to concatenate horizontally all data from different NBA seasons.
    
    Args:
        List of pandas DataFrames.
        
    Returns: 
        Single DataFrame containing the entries for all items in the list of DataFrames. 
    
    """
    all_data = pd.concat(df_list).reset_index().drop('index', axis=1)
    
    return all_data

In [117]:
# Season list 
season_list = [games2013, games2014, games2015, games2016, games2017, games2018, games2019, games2020, games2021]

# Clean each season dataset
for season in season_list:
    get_season(season)
    get_winner(season)
    clean_game_stats(season)

In [118]:
games = combine_seasons(season_list)
games.head()

,Visitor/Neutral,Home/Neutral,Season,H_win
0,Orlando Magic,Indiana Pacers,2013,1
1,Chicago Bulls,Miami Heat,2013,1
2,Los Angeles Clippers,Los Angeles Lakers,2013,1
3,Brooklyn Nets,Cleveland Cavaliers,2013,1
4,Boston Celtics,Toronto Raptors,2013,1


# Exploring team datasets

## Load team stats datasets

In [119]:
stats2013 = pd.read_csv('../data/season_stats/nba_2013_pergamestats.csv')
stats2013.shape

(31, 25)

In [125]:
stats2014 = pd.read_csv('../data/season_stats/nba_2014_pergamestats.csv')
stats2014.shape

(31, 25)

In [126]:
stats2015 = pd.read_csv('../data/season_stats/nba_2015_pergamestats.csv')
stats2015.shape

(31, 25)

In [127]:
stats2016 = pd.read_csv('../data/season_stats/nba_2016_pergamestats.csv')
stats2016.shape

(31, 25)

In [128]:
stats2017 = pd.read_csv('../data/season_stats/nba_2017_pergamestats.csv')
stats2017.shape

(31, 25)

In [129]:
stats2018 = pd.read_csv('../data/season_stats/nba_2018_pergamestats.csv')
stats2018.shape

(31, 25)

In [130]:
stats2019 = pd.read_csv('../data/season_stats/nba_2019_pergamestats.csv')
stats2019.shape

(31, 25)

In [131]:
stats2020 = pd.read_csv('../data/season_stats/nba_2020_pergamestats.csv')
stats2020.shape

(31, 25)

In [132]:
stats2021 = pd.read_csv('../data/season_stats/nba_2021_pergamestats.csv')
stats2021.shape

(31, 25)

## Explore team stats dataset

In [133]:
# What does the dataframe look like? 
stats2013.tail()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
26,27.0,Memphis Grizzlies*,82,241.5,38.1,82.0,0.464,4.9,14.0,0.353,...,0.741,11.6,30.8,42.4,21.9,7.7,4.6,13.7,19.1,96.1
27,28.0,Milwaukee Bucks,82,242.4,36.0,82.2,0.438,6.7,18.9,0.353,...,0.747,11.8,29.3,41.1,21.5,6.6,4.9,15.1,20.9,95.5
28,29.0,Utah Jazz,82,241.2,36.0,81.1,0.444,6.6,19.2,0.344,...,0.747,11.0,30.2,41.2,20.3,7.0,4.5,14.6,20.7,95.0
29,30.0,Chicago Bulls*,82,243.0,34.7,80.2,0.432,6.2,17.8,0.348,...,0.779,11.4,32.7,44.1,22.7,7.2,5.2,14.9,19.1,93.7
30,NaN,League Average,82,242.0,37.7,83.0,0.454,7.7,21.5,0.360,...,0.756,10.9,31.8,42.7,22.0,7.7,4.7,14.6,20.7,101.0


In [134]:
# What are the columns in the dataframe? 
stats2013.columns

Index(['Rk', 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [135]:
# Check for missing values
stats2013.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      30 non-null     float64
 1   Team    31 non-null     object 
 2   G       31 non-null     int64  
 3   MP      31 non-null     float64
 4   FG      31 non-null     float64
 5   FGA     31 non-null     float64
 6   FG%     31 non-null     float64
 7   3P      31 non-null     float64
 8   3PA     31 non-null     float64
 9   3P%     31 non-null     float64
 10  2P      31 non-null     float64
 11  2PA     31 non-null     float64
 12  2P%     31 non-null     float64
 13  FT      31 non-null     float64
 14  FTA     31 non-null     float64
 15  FT%     31 non-null     float64
 16  ORB     31 non-null     float64
 17  DRB     31 non-null     float64
 18  TRB     31 non-null     float64
 19  AST     31 non-null     float64
 20  STL     31 non-null     float64
 21  BLK     31 non-null     float64
 22  TOV 

We can see that there is no missing data. Additionally, we can probably drop the `Rk`, `G` and `MP`. Column pairs `FG` and `FGA`, `3P` and `3PA`, and `FT` and `FTA` are combined to calculate `2P%`, `3P%` and `FT%`, respectively, and can be dropped as well.   

## Cleaning team stats datasets

In [136]:
def format_team_name(df):
    """Format team name to remove '*' that denotes teams which qualified to the playoffs. 
    
    Args:
        Raw pandas DataFrame containing DataFrame['Team'] column with '*' character at the of the team name. 
        The '*' is used to denote teams that qualified to the playoffs during that season.
    
    Returns:
        Clean df['Team'] column without '*'.
    
    """
    df['Team'] = df['Team'].str.replace(r'\*', '', regex=True)
    
    return df

In [137]:
def clean_team_stats(df):
    """Drop unnecessary columns from DataFrames containing team stats in a given season information. Also
    drops rows where df['Team'] == League Average.
    
    Args: 
        Pandas DataFrame containing raw features such as 2P (two-point shots made) 
        and 2PA (two-point shots attempted).
    
    Returns:
        Pandas DataFrame with only the shooting percentages.
    
    """
    df.drop(['Rk', 'G', 'MP', 'FG', 'FGA', '2P', '2PA', '3P', '3PA', 'FT', 'FTA'], axis=1, inplace=True)
    df.drop(df.loc[df['Team'] == 'League Average'].index, inplace=True)
    
    return df

In [138]:
def add_season(df, year):
    """Add season year to a given pandas DataFrame.
    
    Args:
        df: pandas DataFrame without Season information.
        year: int value representing the year of a given NBA season.
        
    Returns:
        A pandas DataFrame with a 'Season' column. 
    
    """
    df['Season'] = year
    
    return df

In [139]:
# Add all dataframes to a list
stats_list = [stats2013, stats2014, stats2015, stats2016, stats2017, stats2018, stats2019, stats2020, stats2021]

# Set the year for the 'add_season' function
year = 2013

# Clean each season dataset
for season in stats_list:
    format_team_name(season)
    clean_team_stats(season)
    add_season(season, year)
    year += 1

In [140]:
stats = combine_seasons(stats_list)
stats.head()

,Team,FG%,3P%,2P%,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
0,Los Angeles Clippers,0.474,0.352,0.525,0.730,10.5,32.5,43.0,24.6,8.6,4.8,13.9,21.5,107.9,2013
1,Houston Rockets,0.472,0.358,0.529,0.712,11.2,34.1,45.3,21.4,7.6,5.6,16.1,20.4,107.7,2013
2,Minnesota Timberwolves,0.444,0.341,0.478,0.778,12.5,32.2,44.7,23.9,8.8,3.6,13.9,18.3,106.9,2013
3,Portland Trail Blazers,0.450,0.372,0.481,0.815,12.5,34.0,46.4,23.2,5.5,4.7,13.7,19.2,106.7,2013
4,Oklahoma City Thunder,0.471,0.361,0.512,0.806,10.8,33.9,44.7,21.9,8.3,6.1,15.3,22.7,106.2,2013


In [141]:
# Total rows should 'n_seasons * n_teams' = 270
stats.shape

(270, 15)

# Combining game results and team stats for each season

In [142]:
games.head()

,Visitor/Neutral,Home/Neutral,Season,H_win
0,Orlando Magic,Indiana Pacers,2013,1
1,Chicago Bulls,Miami Heat,2013,1
2,Los Angeles Clippers,Los Angeles Lakers,2013,1
3,Brooklyn Nets,Cleveland Cavaliers,2013,1
4,Boston Celtics,Toronto Raptors,2013,1


In [143]:
stats.head()

,Team,FG%,3P%,2P%,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
0,Los Angeles Clippers,0.474,0.352,0.525,0.730,10.5,32.5,43.0,24.6,8.6,4.8,13.9,21.5,107.9,2013
1,Houston Rockets,0.472,0.358,0.529,0.712,11.2,34.1,45.3,21.4,7.6,5.6,16.1,20.4,107.7,2013
2,Minnesota Timberwolves,0.444,0.341,0.478,0.778,12.5,32.2,44.7,23.9,8.8,3.6,13.9,18.3,106.9,2013
3,Portland Trail Blazers,0.450,0.372,0.481,0.815,12.5,34.0,46.4,23.2,5.5,4.7,13.7,19.2,106.7,2013
4,Oklahoma City Thunder,0.471,0.361,0.512,0.806,10.8,33.9,44.7,21.9,8.3,6.1,15.3,22.7,106.2,2013


In [144]:
# Put all of the data into a single 'merged_df'
merged_df = games.merge(stats, how='left', left_on=['Visitor/Neutral', 'Season'], right_on=['Team', 'Season'])
merged_df = merged_df.merge(stats, how='left', left_on=['Home/Neutral', 'Season'], right_on=['Team', 'Season'])
merged_df.head()

# Look at sklearn polynomial features
# Look at PCA to reduce dimensionality and other tools

# Rename columns 
#test_df.columns = []

,Visitor/Neutral,Home/Neutral,Season,H_win,Team_x,FG%_x,3P%_x,2P%_x,FT%_x,ORB_x,...,FT%_y,ORB_y,DRB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PTS_y
0,Orlando Magic,Indiana Pacers,2013,1,Orlando Magic,0.445,0.353,0.474,0.763,9.7,...,0.779,10.2,34.5,44.7,20.1,6.7,5.4,15.1,20.4,96.7
1,Chicago Bulls,Miami Heat,2013,1,Chicago Bulls,0.432,0.348,0.456,0.779,11.4,...,0.760,7.6,29.2,36.9,22.5,8.9,4.5,14.8,19.5,102.2
2,Los Angeles Clippers,Los Angeles Lakers,2013,1,Los Angeles Clippers,0.474,0.352,0.525,0.730,10.5,...,0.757,9.1,32.0,41.0,24.5,7.5,5.4,15.1,19.8,103.0
3,Brooklyn Nets,Cleveland Cavaliers,2013,1,Brooklyn Nets,0.459,0.369,0.497,0.753,8.8,...,0.751,12.1,32.1,44.1,21.2,7.1,3.7,14.2,20.0,98.2
4,Boston Celtics,Toronto Raptors,2013,1,Boston Celtics,0.435,0.333,0.470,0.777,12.0,...,0.782,11.4,31.1,42.5,21.2,7.0,4.2,14.1,23.0,101.3


We can see that our final DataFrame has a lot of features. We should consider:
* Dropping columns/features that are not necessary.
* Look at sklearn polynomial features.
* Look at PCA to reduce dimensionality and other tools.

In [146]:
# Drop columns with team names
#clean_df = merged_df.drop(['Visitor/Neutral', 'Home/Neutral', 'Team_x', 'Team_y'], axis=1)
clean_df

,Season,H_win,FG%_x,3P%_x,2P%_x,FT%_x,ORB_x,DRB_x,TRB_x,AST_x,...,FT%_y,ORB_y,DRB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PTS_y
0,2013,1,0.445,0.353,0.474,0.763,9.7,32.4,42.0,21.0,...,0.779,10.2,34.5,44.7,20.1,6.7,5.4,15.1,20.4,96.7
1,2013,1,0.432,0.348,0.456,0.779,11.4,32.7,44.1,22.7,...,0.760,7.6,29.2,36.9,22.5,8.9,4.5,14.8,19.5,102.2
2,2013,1,0.474,0.352,0.525,0.730,10.5,32.5,43.0,24.6,...,0.757,9.1,32.0,41.0,24.5,7.5,5.4,15.1,19.8,103.0
3,2013,1,0.459,0.369,0.497,0.753,8.8,29.4,38.1,20.9,...,0.751,12.1,32.1,44.1,21.2,7.1,3.7,14.2,20.0,98.2
4,2013,1,0.435,0.333,0.470,0.777,12.0,30.5,42.5,21.0,...,0.782,11.4,31.1,42.5,21.2,7.0,4.2,14.1,23.0,101.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10744,2021,0,0.469,0.347,0.546,0.732,9.5,34.5,44.0,24.0,...,0.795,9.2,34.9,44.1,27.8,7.2,3.7,14.5,20.0,112.7
10745,2021,1,0.430,0.323,0.507,0.756,10.4,35.2,45.6,22.2,...,0.793,9.1,34.9,44.0,24.0,7.4,5.0,13.7,18.6,108.4
10746,2021,0,0.469,0.364,0.557,0.769,9.8,35.7,45.5,27.1,...,0.789,12.0,33.2,45.2,25.0,8.3,4.0,14.1,19.7,109.3
10747,2021,0,0.460,0.344,0.530,0.768,9.6,33.4,42.9,23.7,...,0.797,9.8,35.5,45.3,27.4,8.6,4.4,12.9,19.9,114.8


In [148]:
# Rename columns using V and H to represent visiting and home team, respectively
clean_df.columns = ['Season', 'H_win', 'FG%_V', '3P%_V', '2P%_V', 'FT%_V', 'ORB_V', 'DRB_V',
       'TRB_V', 'AST_V', 'STL_V', 'BLK_V', 'TOV_V', 'PF_V', 'PTS_V', 'FG%_H',
       '3P%_H', '2P%_H', 'FT%_H', 'ORB_H', 'DRB_H', 'TRB_H', 'AST_H', 'STL_H',
       'BLK_H', 'TOV_H', 'PF_H', 'PTS_H']

In [149]:
clean_df

,Season,H_win,FG%_V,3P%_V,2P%_V,FT%_V,ORB_V,DRB_V,TRB_V,AST_V,...,FT%_H,ORB_H,DRB_H,TRB_H,AST_H,STL_H,BLK_H,TOV_H,PF_H,PTS_H
0,2013,1,0.445,0.353,0.474,0.763,9.7,32.4,42.0,21.0,...,0.779,10.2,34.5,44.7,20.1,6.7,5.4,15.1,20.4,96.7
1,2013,1,0.432,0.348,0.456,0.779,11.4,32.7,44.1,22.7,...,0.760,7.6,29.2,36.9,22.5,8.9,4.5,14.8,19.5,102.2
2,2013,1,0.474,0.352,0.525,0.730,10.5,32.5,43.0,24.6,...,0.757,9.1,32.0,41.0,24.5,7.5,5.4,15.1,19.8,103.0
3,2013,1,0.459,0.369,0.497,0.753,8.8,29.4,38.1,20.9,...,0.751,12.1,32.1,44.1,21.2,7.1,3.7,14.2,20.0,98.2
4,2013,1,0.435,0.333,0.470,0.777,12.0,30.5,42.5,21.0,...,0.782,11.4,31.1,42.5,21.2,7.0,4.2,14.1,23.0,101.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10744,2021,0,0.469,0.347,0.546,0.732,9.5,34.5,44.0,24.0,...,0.795,9.2,34.9,44.1,27.8,7.2,3.7,14.5,20.0,112.7
10745,2021,1,0.430,0.323,0.507,0.756,10.4,35.2,45.6,22.2,...,0.793,9.1,34.9,44.0,24.0,7.4,5.0,13.7,18.6,108.4
10746,2021,0,0.469,0.364,0.557,0.769,9.8,35.7,45.5,27.1,...,0.789,12.0,33.2,45.2,25.0,8.3,4.0,14.1,19.7,109.3
10747,2021,0,0.460,0.344,0.530,0.768,9.6,33.4,42.9,23.7,...,0.797,9.8,35.5,45.3,27.4,8.6,4.4,12.9,19.9,114.8


# Exporting final dataset with game results and team data

In [150]:
clean_df.to_csv('../data/nba_data.csv', index=False)